https://dipbigdata.kr/pages/sub.htm?mode=view&mv_data=aWR4PTQ0JnN0YXJ0UGFnZT0wJmxpc3RObz0yMyZ0YWJsZT1jc19vbmxpbmVfZGF0YSZzaXRlX2NvZGU9ZGlwJm5hdl9jb2RlPWRpcDE2NjY3NzE5OTYmY29kZT0mc2VhcmNoX2l0ZW09JnNlYXJjaF9vcmRlcj0mb3JkZXJfbGlzdD0mbGlzdF9zY2FsZT0mdmlld19sZXZlbD0mdmlld19jYXRlPeu5heuNsOydtO2EsOu2hOyEneq4sOyCrCZ2aWV3X2NhdGUyPQ==||

회귀예측 문제

In [1]:
# 데이터 가져오기
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = sns.load_dataset('mpg')
X_train,X_test,y_train,y_test = train_test_split(df,df['mpg'],test_size=0.2,random_state=42)

X_train = X_train.drop(['mpg'],axis=1)
X_test = X_test.drop(['mpg'],axis=1)

In [2]:
X_train.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
3,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
18,4,97.0,88.0,2130,14.5,70,japan,datsun pl510
376,4,91.0,68.0,2025,18.2,82,japan,mazda glc custom l
248,4,91.0,60.0,1800,16.4,78,japan,honda civic cvcc
177,4,115.0,95.0,2694,15.0,75,europe,audi 100ls


# 1. 결측치

In [3]:
X_train.isna().sum()

cylinders       0
displacement    0
horsepower      5
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

In [5]:
# horsepower 중앙값으로 채우기
X_train['horsepower'] = X_train['horsepower'].fillna(X_train['horsepower'].median())
X_test['horsepower'] = X_test['horsepower'].fillna(X_test['horsepower'].median())

In [6]:
X_train.isna().sum()

cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

In [7]:
X_test.isna().sum()

cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

# 2. 라벨 인코더

In [10]:
X_train.head(1)

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
3,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst


In [11]:
label = ['origin','name']
from sklearn.preprocessing import LabelEncoder
X_train[label] = X_train[label].apply(LabelEncoder().fit_transform)
X_test[label] = X_test[label].apply(LabelEncoder().fit_transform)

In [13]:
X_train.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
3,8,304.0,150.0,3433,12.0,70,2,10
18,4,97.0,88.0,2130,14.5,70,1,78
376,4,91.0,68.0,2025,18.2,82,1,149
248,4,91.0,60.0,1800,16.4,78,1,143
177,4,115.0,95.0,2694,15.0,75,0,13


In [14]:
X_train.origin.value_counts()

origin
2    196
1     66
0     56
Name: count, dtype: int64

In [15]:
X_train.name.value_counts()

name
7      5
217    5
129    5
175    4
6      4
      ..
185    1
16     1
113    1
166    1
236    1
Name: count, Length: 246, dtype: int64

# 3. 카테고리 변환,더미

In [16]:
# origin
X_train['origin'] = X_train['origin'].astype('category')
X_test['origin'] = X_test['origin'].astype('category')

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [17]:
X_train.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,name,origin_0,origin_1,origin_2
3,8,304.0,150.0,3433,12.0,70,10,False,False,True
18,4,97.0,88.0,2130,14.5,70,78,False,True,False
376,4,91.0,68.0,2025,18.2,82,149,False,True,False
248,4,91.0,60.0,1800,16.4,78,143,False,True,False
177,4,115.0,95.0,2694,15.0,75,13,True,False,False


# 4. 파생 변수

In [18]:
# horsepower
X_train['horsepower_qcut'] = pd.qcut(X_train['horsepower'],5,labels=False)
X_test['horsepower_qcut'] = pd.qcut(X_test['horsepower'],5,labels=False)


In [19]:
X_train.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,name,origin_0,origin_1,origin_2,horsepower_qcut
3,8,304.0,150.0,3433,12.0,70,10,False,False,True,4
18,4,97.0,88.0,2130,14.5,70,78,False,True,False,1
376,4,91.0,68.0,2025,18.2,82,149,False,True,False,0
248,4,91.0,60.0,1800,16.4,78,143,False,True,False,0
177,4,115.0,95.0,2694,15.0,75,13,True,False,False,2


# 5. 스케일

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = ['displacement', 'horsepower', 'weight']
m = MinMaxScaler()
m.fit(X_train[scaler])

X_train[scaler] = m.transform(X_train[scaler])
X_test[scaler] = m.transform(X_test[scaler])

In [21]:
X_train.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,name,origin_0,origin_1,origin_2,horsepower_qcut
3,8,0.609819,0.581006,0.516019,12.0,70,10,False,False,True,4
18,4,0.074935,0.234637,0.146583,14.5,70,78,False,True,False,1
376,4,0.059432,0.122905,0.116813,18.2,82,149,False,True,False,0
248,4,0.059432,0.078212,0.053020,16.4,78,143,False,True,False,0
177,4,0.121447,0.273743,0.306493,15.0,75,13,True,False,False,2


# 6. 데이터 분리 (학습, 검증)

In [22]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state=42)

print(X_train.shape)
print(X_valid.shape)

(254, 11)
(64, 11)


# 7. 모형 학습

In [23]:
from sklearn.linear_model import LinearRegression
model1 = LinearRegression()
model1.fit(X_train, y_train)
pred1 = model1.predict(X_valid)

In [24]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor()
model2.fit(X_train, y_train)
pred2 = model2.predict(X_valid)

# 8. 앙상블

In [25]:
from sklearn.ensemble import StackingRegressor
estimators = [('lr',model1),('rf',model2)]
model3 = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor())
model3.fit(X_train,y_train)
pred3 = model3.predict(X_valid)

# 9. 모형 평가

In [26]:
from sklearn.metrics import mean_squared_error

print('선형',mean_squared_error(y_valid,pred1))
print('랜포',mean_squared_error(y_valid,pred2))
print('스태킹',mean_squared_error(y_valid,pred3))

선형 12.966610337469966
랜포 9.650148156250001
스태킹 11.19776651562501


# 10. 하이퍼파라미터 튜닝

In [27]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators' : [50,100],
              'max_depth' : [4,6]}
model5 = RandomForestRegressor()
clf = GridSearchCV(estimator=model5, param_grid = parameters,cv=3)
clf.fit(X_train,y_train)
print('최적',clf.best_params_)

최적 {'max_depth': 6, 'n_estimators': 50}


# 11. 파일 저장

In [28]:
result = pd.DataFrame(model2.predict(X_test))
result.head()

,0
0,30.185
1,29.562
2,20.795
3,15.320
4,14.449


In [31]:
result = result.iloc[:,0]

In [32]:
pd.DataFrame({'id':X_test.index,'result':result})

,id,result
0,198,30.185
1,396,29.562
2,33,20.795
3,208,15.320
4,93,14.449
...,...,...
75,249,20.408
76,225,18.843
77,367,28.703
78,175,29.100


In [33]:
pd.DataFrame({'id':X_test.index,'result':result}).to_csv('연습문제2결과.csv',index=False)